### assess scores using mubind

In [50]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
import mubind
import pandas as pd
import glob
import os
import anndata


In [52]:
cd ~/workspace/theislab/mubind-pipeline/notebooks/pipeline/

/home/ilibarra/workspace/theislab/mubind-pipeline/notebooks/pipeline


In [53]:
import utils

In [54]:
import os
import datetime
def old_file(path, n_hours):
    assert os.path.exists(path)
    mod_time = os.path.getctime(path)
    return datetime.fromtimestamp(mod_time) > datetime.now() - datetime.timedelta(hours=n_hours)

In [55]:
import torch
import mubind as mb
import numpy as np
import pickle
import sklearn
import datetime
# rootdir = '/mnt/f/workspace/theislab/mubind/data/*/*/mubind_output/obs*/*/*'
rootdir = '/home/ilibarra/workspace/theislab/mubind/data/*/*/mubind_output/obs*/*/*/*'

overwrite = False

df = []
for p in glob.glob(rootdir):
    # print(p)
    if 'obs100/' in p:
        continue
    if not 'model_0.pt' in p and not 'model_1.pt' in p:
        continue
    # if not 'model_1.pt' in p:
    #     continue
        
    # filename = os.path.basename(p)
    # print(filename)
    # continue
    runtime_path = p.replace('model_', 'running_time_').replace('.pt', '.pkl')
    time_diff = pickle.load(open(runtime_path, 'rb')) if os.path.exists(runtime_path) else None

    print(time_diff, type(time_diff))
    if not isinstance(time_diff, datetime.timedelta):
        print('time log not ok....skip')
        continue
        
    scores_path = p.replace('.pt', '_scores.csv')
    print(os.path.exists(scores_path), scores_path)

    # if scores_path != '/home/ilibarra/workspace/theislab/mubind/data/pancreatic_endocrinogenesis/random/mubind_output/obs10000/fold_0/pwms/model_0_scores.csv':
    #     continue

    # assert False
    
    if os.path.exists(scores_path) and not overwrite:
        df2 = pd.read_csv(scores_path, index_col=0)
        df2['n_obs'] = int(scores_path.split('/')[-4].replace('obs', ''))
        df2['feat_selection'] = scores_path.split('/')[-6].replace('obs', '')
        df.append(df2)
        continue

    # continue



    # 
    print('loading model')
    res = []

    model = torch.load(p)# .to('cpu')

    feature_selection_rule = p.split('/')[-6]
    model_type = p.split('/')[-3]

    os.makedirs("results", exist_ok=True)

    fold_key = p.split('/')[-2]


    dataset_name = p.split('/')[-7]
    n_obs = p.split('/')[-4]

    print(n_obs, dataset_name, fold_key, model_type)
    # assert False

    ad_path = os.path.join(os.path.dirname(p),
                           '../../../../scbasset_input/%s/%s/ad.h5ad' % (n_obs, fold_key))
    splits_path = os.path.join(os.path.dirname(p),
                               '../../../../scbasset_input/%s/%s/splits.h5' % (n_obs, fold_key))
    peaks_path = os.path.join(os.path.dirname(p),
                              '../../../../scbasset_input/%s/%s/peaks.bed' % (n_obs, fold_key))
    
    print(os.path.exists(ad_path), ad_path)
    print(os.path.exists(splits_path), splits_path)
    print(os.path.exists(peaks_path), peaks_path)

    # read h5ad file
    ad = anndata.read_h5ad(ad_path)

    # print('preparing sequences test...')
    # print(dataset_name)

    species = 'hg38' if (dataset_name != 'noack_2022' and dataset_name != 'pancreatic_endocrinogenesis') else 'mm10'
    # gen_path = '/mnt/c/Users/IgnacioIbarra/Dropbox/annotations/%s/genome/%s.fa' % (species, species)
    gen_path = '/home/ilibarra/annotations/%s/genome/%s.fa' % (species, species)

    seqs = pd.read_csv(peaks_path, sep='\t', header=None)
    seqs.columns = ['chr', 'start', 'end']

    assert seqs.shape[0] == ad.shape[1]
    # print(seqs.shape, ad.shape)

    # fix seq coordinates
    ad.var['chr'] = np.where(ad.var_names.str.contains(':'), ad.var_names.str.split(':').str[0], ad.var_names.str.split('-').str[0])
    ad.var['chr'] = np.where(~ad.var['chr'].str.contains('chr'), 'chr', '') + ad.var['chr']
    ad.var['start'] = [s.split(':')[1].split('-')[0] if ':' in s else s.split('-')[1] for s in ad.var_names]
    # ad.var['start'] = np.where(~ad.var_names.str.contains(':'), ad.var_names.str.split('-').str[1], ad.var_names.str.split(':').str[1].str.split('-').str[0])
    ad.var['end'] = np.where(~ad.var_names.str.contains(':'), ad.var_names.str.split('-').str[2], ad.var_names.str.split('-').str[1])

    seqs = ad.var[['chr', 'start', 'end']]

    all_seqs = mb.bindome.tl.get_sequences_from_bed(seqs[['chr', 'start', 'end']],
                                                    gen_path=gen_path,
                                                    bin='/home/ilibarra/miniconda3/envs/mubind/bin/bedtools',
                                                    uppercase=True)
                                                # gen_path='../../../annotations/dm6/genome/dm6.fa')
    keys_test = set([s[0] for s in all_seqs])
    # ad_test = ad_test[:,ad_test.var['k_summit'].isin(keys_test)]

    # convert to csr matrix
    import h5py
    with h5py.File(splits_path, 'r') as hf:
        train_ids = hf['train_ids'][:]
        val_ids = hf['val_ids'][:]
        test_ids = hf['test_ids'][:]


    assert ad.shape[1] == len(all_seqs)
    # print(ad_train.shape, ad_test.shape)


    import numpy as np
    import torch.utils.data as tdata

    res = []

    ad_train = ad[:,train_ids]
    ad_val = ad[:,val_ids]
    ad_test = ad[:,test_ids]
    ad_train.shape, ad_val.shape, ad_test.shape


    all_seqs = np.array(all_seqs)
    train_seqs, val_seqs, test_seqs = all_seqs[train_ids], all_seqs[val_ids], all_seqs[test_ids]

    for next_ad, next_seqs, next_label in zip([ad_train, ad_val, ad_test],
                                            [train_seqs, val_seqs, test_seqs],
                                            ['train', 'val', 'test']):
        next_df = utils.prepare_df(next_seqs, next_ad)

        dataset = mb.datasets.SelexDataset(next_df,
                                                n_rounds=next_df.shape[1],
                                                enr_series=False)
        dataloader = tdata.DataLoader(dataset=dataset,
                                batch_size=512,
                                shuffle=True)


        try:
            roc_auc, pr_auc, pr_auc_multi, roc_auc_multi = np.nan, np.nan, np.nan, np.nan
            scores = {'r2_counts': np.nan}
            # try:
            print('prauc_multi')
            pr_auc_multi = utils.get_auprc_multiclass(model, dataloader)
            print('roc_multi')
            roc_auc_multi = utils.get_auroc_multiclass(model, dataloader)

            print('auprc')
            pr_auc = utils.get_auprc(model, dataloader)
            print('roc')
            roc_auc = utils.get_auroc(model, dataloader)

            print(pr_auc, pr_auc_multi)
            
            scores = mb.tl.scores(model, dataloader)

            print('here...')
            # except:
            #     print('problem with fold', fold_key)
            #     continue
            # assert False

            dt = time_diff.total_seconds()
            res.append([dataset_name, next_label, feature_selection_rule, 'roc_auc', roc_auc, dt, p, fold_key, model_type])
            res.append([dataset_name, next_label, feature_selection_rule, 'pr_auc', pr_auc, dt, p, fold_key, model_type])
            res.append([dataset_name, next_label, feature_selection_rule, 'r2', scores['r2_counts'], dt, p, fold_key, model_type])

            res.append([dataset_name, next_label, feature_selection_rule, 'roc_auc_multi', roc_auc_multi, dt, p, fold_key, model_type])
            res.append([dataset_name, next_label, feature_selection_rule, 'pr_auc_multi', pr_auc_multi, dt, p, fold_key, model_type])
        except:
            print('problem with fold',  fold_key)

        
    res = pd.DataFrame(res, columns=['dataset', 'group', 'feat_selection', 'metric', 'value', 'time', 'model_path', 'fold_key', 'model_type'])

    res['n_obs'] = int(scores_path.split('/')[-4].replace('obs', ''))
    res.to_csv(scores_path)
    
    df.append(res)

    # assert False


0:09:07.116369 <class 'datetime.timedelta'>
False /home/ilibarra/workspace/theislab/mubind/data/pbmc/random/mubind_output/obs2000/pwms/fold_0/model_1_scores.csv
loading model
obs2000 pbmc fold_0 pwms
True /home/ilibarra/workspace/theislab/mubind/data/pbmc/random/mubind_output/obs2000/pwms/fold_0/../../../../scbasset_input/obs2000/fold_0/ad.h5ad
True /home/ilibarra/workspace/theislab/mubind/data/pbmc/random/mubind_output/obs2000/pwms/fold_0/../../../../scbasset_input/obs2000/fold_0/splits.h5
True /home/ilibarra/workspace/theislab/mubind/data/pbmc/random/mubind_output/obs2000/pwms/fold_0/../../../../scbasset_input/obs2000/fold_0/peaks.bed
/tmp/tmprhnyh0k7
genome hg19 False
/home/ilibarra/annotations/hg38/genome/hg38.fa
True /home/ilibarra/annotations/hg38/genome/hg38.fa
running bedtools...
['/home/ilibarra/miniconda3/envs/mubind/bin/bedtools', 'getfasta', '-fi', '/home/ilibarra/annotations/hg38/genome/hg38.fa', '-bed', '/tmp/tmprhnyh0k7', '-fo', '/tmp/tmp_4hwnhww']
/home/ilibarra/minicon

In [56]:
res = pd.concat(df)
res['model'] = np.where(res['model_path'].str.contains('0.pt'), 'mubind', 'mubind_graph')
print('')

In [57]:
res

,dataset,group,feat_selection,metric,value,time,model_path,fold_key,model_type,n_obs,model
0,pbmc,val,random,roc_auc,0.541125,547.116369,/home/ilibarra/workspace/theislab/mubind/data/...,fold_0,pwms,2000,mubind_graph
1,pbmc,val,random,pr_auc,0.070541,547.116369,/home/ilibarra/workspace/theislab/mubind/data/...,fold_0,pwms,2000,mubind_graph
2,pbmc,val,random,r2,0.324882,547.116369,/home/ilibarra/workspace/theislab/mubind/data/...,fold_0,pwms,2000,mubind_graph
3,pbmc,val,random,roc_auc_multi,0.499715,547.116369,/home/ilibarra/workspace/theislab/mubind/data/...,fold_0,pwms,2000,mubind_graph
4,pbmc,val,random,pr_auc_multi,0.083308,547.116369,/home/ilibarra/workspace/theislab/mubind/data/...,fold_0,pwms,2000,mubind_graph
...,...,...,...,...,...,...,...,...,...,...,...
10,noack_2022,test,episcanpy,roc_auc,0.626203,34.548246,/home/ilibarra/workspace/theislab/mubind/data/...,fold_3,pwms,1000,mubind
11,noack_2022,test,episcanpy,pr_auc,0.009562,34.548246,/home/ilibarra/workspace/theislab/mubind/data/...,fold_3,pwms,1000,mubind
12,noack_2022,test,episcanpy,r2,0.001799,34.548246,/home/ilibarra/workspace/theislab/mubind/data/...,fold_3,pwms,1000,mubind
13,noack_2022,test,episcanpy,roc_auc_multi,0.500000,34.548246,/home/ilibarra/workspace/theislab/mubind/data/...,fold_3,pwms,1000,mubind


In [58]:
res['model'].value_counts()

model
mubind_graph    1375
mubind          1360
Name: count, dtype: int64

In [59]:
res.to_csv('results_mubind.csv')